<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-29 14:16:24
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.34 C
-------------------
Today PnL: 69.61 K (0.52%)
Current PnL: -17.57 L (-12.24%)
CY Booked + Current PnL: -9.49 L (-6.61%)
-------------------
Total profit:  2.76 L
Total loss:  -20.33 L
-------------------
Total Booked + Current PnL: 17.49 L (15.01%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.05%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 75.78 L (56.72%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 8.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-222.73,H-SC,12.17,88530.0,-6786.0,6826.0,2.03,-7.12,7.71,0.04,161.0,-0.99,0.66,74.62,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.41,110784.0,6472.0,11067.0,0.44,6.20,9.99,16.81,10.0,0.58,0.83,23.25,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-17.85,H-LC,6.72,111855.0,8289.0,12416.0,1.37,8.00,11.10,20.00,14.0,0.67,0.84,23.20,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,12.87,H-LC,9.68,101349.0,7916.0,14675.0,0.11,8.47,14.48,24.18,35.0,0.54,0.76,23.07,XY25,ATH,FMCG
70,SBIN,760.30,863.00,-12.46,M-LC,4.41,211881.0,10402.0,16823.0,0.30,5.16,7.94,13.51,60.0,0.62,1.59,18.41,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-33.88,M-SC,42.08,65525.0,519.0,150413.0,1.80,0.80,229.55,232.18,196.0,0.00,0.49,1.80,XY24,NTT,MISC
53,KPIGREEN,497.21,731.64,23.50,H-SC,12.62,126202.0,905.0,58167.0,-3.66,0.72,46.09,47.15,143.0,0.02,0.95,57.89,MH,ATH,POWER
14,BERGEPAINT,561.33,680.00,-19.92,H-MC,11.14,226840.0,-499.0,48566.0,0.38,-0.22,21.41,21.14,108.0,-0.01,1.70,27.01,XY24,NTT,PAINTS
38,HEROMOTOCO,4311.81,5949.07,-5.71,H-MC,3.16,151462.0,549.0,56753.0,0.85,0.36,37.47,37.97,89.0,0.01,1.13,25.76,AR,ATH,AUTO
59,NATIONALUM,189.63,247.44,-50.15,H-MC,3.55,100144.0,-360.0,31005.0,0.67,-0.36,30.96,30.49,81.0,-0.01,0.75,31.93,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-18.08,H-LC,1.37,215241.0,10741.0,30672.0,-0.06,5.25,14.25,20.25,7.0,0.35,1.61,14.44,X40N,BTT,PHARMA
56,LTIM,5564.16,7230.2,-5.08,H-LC,2.53,182754.0,-17556.0,77524.0,0.58,-8.76,42.42,29.94,26.0,-0.23,1.37,26.26,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-8.57,H-LC,2.74,189682.0,-10824.0,59598.0,0.40,-5.40,31.42,24.33,30.0,-0.18,1.42,9.38,X40N,BTT,FINANCE
40,HINDUNILVR,2413.81,2723.0,-12.33,H-LC,3.42,274411.0,4064.0,30569.0,0.35,1.50,11.14,12.81,18.0,0.13,2.06,14.76,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-17.94,H-LC,3.50,163071.0,-38315.0,125923.0,1.63,-19.03,77.22,43.51,20.0,-0.30,1.22,13.51,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,20.86,71021.0,-42528.0,82576.0,6.66,-37.45,116.27,35.27,267.0,-0.52,0.53,73.83,XR,NTT,HOTELS
18,BSOFT,628.85,836.99,-15.15,M-SC,12.37,71172.0,-38877.0,75300.0,5.57,-35.33,105.80,33.10,216.0,-0.52,0.53,18.26,XR,ATH,IT
34,HAPPSTMNDS,919.80,1488.71,-5.06,M-SC,7.60,66182.0,-30397.0,90133.0,4.83,-31.47,136.19,61.85,217.0,-0.34,0.50,16.69,AR,ATH,IT
79,TANLA,917.30,1963.11,-31.70,H-SC,6.70,167040.0,-72375.0,345338.0,4.21,-30.23,206.74,114.01,130.0,-0.21,1.25,53.20,AR,ATH,IT
91,VBL,492.64,672.28,-10.17,H-LC,12.46,272551.0,7511.0,89124.0,4.08,2.83,32.70,36.46,2.0,0.08,2.04,16.40,X40N,ATH,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-311.48,H-SC,21.13,89947.0,-43339.0,127932.0,-5.00,-32.52,142.23,63.47,150.0,-0.34,0.67,50.77,XR,NTT,CHEMICALS
53,KPIGREEN,497.21,731.64,23.50,H-SC,12.62,126202.0,905.0,58167.0,-3.66,0.72,46.09,47.15,143.0,0.02,0.95,57.89,MH,ATH,POWER
57,MASFIN,326.60,399.50,-18.25,H-SC,16.77,93030.0,-4950.0,26821.0,-2.16,-5.05,28.83,22.32,149.0,-0.18,0.70,34.93,XR,ATH,FINANCE
58,MEDANTA,1087.93,1486.00,20.18,H-SC,11.22,153967.0,27767.0,18414.0,-2.07,22.00,11.96,36.59,154.0,1.51,1.15,37.27,XY24,NTT,HEALTHCARE
31,EASEMYTRIP,18.12,26.40,1.63,M-SC,19.78,97873.0,-78942.0,159739.0,-1.95,-44.65,163.21,45.70,195.0,-0.49,0.73,0.00,XY24,NTT,TRAVEL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-222.73,H-SC,12.17,88530.0,-6786.0,6826.0,2.03,-7.12,7.71,0.04,161.0,-0.99,0.66,74.62,OX40N,NTT,MISC
78,SYMPHONY,1306.42,1306.0,-30.70,M-SC,9.47,147572.0,-23569.0,23508.0,1.60,-13.77,15.93,-0.03,194.0,-1.00,1.11,6.79,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,121.54,H-SC,7.05,145378.0,-29181.0,29221.0,2.63,-16.72,20.10,0.02,142.0,-1.00,1.09,26.06,OX40N,NTT,PAINTS
86,UJJIVANSFB,52.77,53.0,36.52,M-SC,21.74,118125.0,-24354.0,24972.0,-0.09,-17.09,21.14,0.44,249.0,-0.98,0.88,39.42,OX40N,NTT,BANKS
80,TATAELXSI,7332.28,7332.0,-19.33,H-MC,1.72,84679.0,-17973.0,17969.0,0.22,-17.51,21.22,-0.00,100.0,-1.00,0.63,28.72,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.48,75265.0,40816.0,4155.0,-0.22,118.48,5.52,130.54,1.0,9.82,0.56,15.09,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-10.17,H-LC,12.46,272551.0,7511.0,89124.0,4.08,2.83,32.70,36.46,2.0,0.08,2.04,16.40,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-26.74,H-LC,12.74,238524.0,-57410.0,151320.0,-0.69,-19.40,63.44,31.73,3.0,-0.38,1.79,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-15.29,H-LC,6.88,271364.0,9763.0,135872.0,0.02,3.73,50.07,55.67,5.0,0.07,2.03,10.06,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.18,H-LC,7.38,217210.0,691.0,92206.0,0.43,0.32,42.45,42.91,6.0,0.01,1.63,13.54,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,3.27,53436.0,12181.0,5253.0,2.43,29.53,9.83,42.26,197.0,2.32,0.40,25.81,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,20.18,H-SC,11.22,153967.0,27767.0,18414.0,-2.07,22.00,11.96,36.59,154.0,1.51,1.15,37.27,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.48,75265.0,40816.0,4155.0,-0.22,118.48,5.52,130.54,1.0,9.82,0.56,15.09,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,23.50,H-SC,12.62,126202.0,905.0,58167.0,-3.66,0.72,46.09,47.15,143.0,0.02,0.95,57.89,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,20.18,H-SC,11.22,153967.0,27767.0,18414.0,-2.07,22.00,11.96,36.59,154.0,1.51,1.15,37.27,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,-2.80,H-SC,0.72,109231.0,7227.0,37117.0,0.13,7.09,33.98,43.48,122.0,0.19,0.82,25.05,X40N,ATH,MISC
38,HEROMOTOCO,4311.81,5949.07,-5.71,H-MC,3.16,151462.0,549.0,56753.0,0.85,0.36,37.47,37.97,89.0,0.01,1.13,25.76,AR,ATH,AUTO
26,DABUR,505.20,735.00,-6.82,H-MC,4.51,201864.0,4836.0,84783.0,-0.84,2.45,42.00,45.49,96.0,0.06,1.51,15.81,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.48,75265.0,40816.0,4155.0,-0.22,118.48,5.52,130.54,1.0,9.82,0.56,15.09,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-38.05,M-SC,8.40,104242.0,12744.0,66006.0,0.74,13.93,63.32,86.07,205.0,0.19,0.78,50.80,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-47.89,H-SC,12.85,135680.0,12344.0,121420.0,0.14,10.01,89.49,108.45,118.0,0.10,1.02,34.60,AR,ATH,MISC
95,WIPRO,243.46,420.00,-9.21,M-LC,5.63,155837.0,4892.0,104567.0,0.52,3.24,67.10,72.51,56.0,0.05,1.17,9.46,XR,NTT,IT
53,KPIGREEN,497.21,731.64,23.50,H-SC,12.62,126202.0,905.0,58167.0,-3.66,0.72,46.09,47.15,143.0,0.02,0.95,57.89,MH,ATH,POWER


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,6.70,20242.0,-29351.0,553.0,3.59,-59.18,2.73,-58.07,204.0,-53.08,0.15,35.75,XY24,ATH,MISC
28,DIGITIDE,188.38,267.99,NaN,NaN,3.27,53436.0,12181.0,5253.0,2.43,29.53,9.83,42.26,197.0,2.32,0.40,25.81,XY24,ATH,IT
74,SIS,477.00,477.00,2288.60,M-SC,5.83,64260.0,-16830.0,16830.0,0.73,-20.75,26.19,0.00,238.0,-1.00,0.48,30.23,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,20.86,71021.0,-42528.0,82576.0,6.66,-37.45,116.27,35.27,267.0,-0.52,0.53,73.83,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-222.73,H-SC,12.17,88530.0,-6786.0,6826.0,2.03,-7.12,7.71,0.04,161.0,-0.99,0.66,74.62,OX40N,NTT,MISC


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,3.27,53436.0,12181.0,5253.0,2.43,29.53,9.83,42.26,197.0,2.32,0.40,25.81,XY24,ATH,IT
16,BLUSPRING,226.45,94.95,NaN,NaN,6.70,20242.0,-29351.0,553.0,3.59,-59.18,2.73,-58.07,204.0,-53.08,0.15,35.75,XY24,ATH,MISC
33,GREENPANEL,375.16,537.00,257.24,M-SC,9.39,159576.0,-23502.0,102480.0,2.11,-12.84,64.22,43.14,234.0,-0.23,1.20,50.10,XY24,NTT,MISC
58,MEDANTA,1087.93,1486.00,20.18,H-SC,11.22,153967.0,27767.0,18414.0,-2.07,22.00,11.96,36.59,154.0,1.51,1.15,37.27,XY24,NTT,HEALTHCARE
72,SHALBY,261.39,327.00,976.56,M-SC,9.64,146500.0,-34643.0,80106.0,0.84,-19.12,54.68,25.10,228.0,-0.43,1.10,17.41,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.53
1,25,41.24
2,50,70.21


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.62
LC    35.26
MC    21.96
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.79
X40N     14.66
X40      12.50
XY25     10.78
XR        9.08
OX40N     8.39
AR        8.27
X5K       2.25
MH        1.70
X200      1.37
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.01
H-SC    25.14
H-MC    19.00
M-SC    15.06
M-LC     7.19
M-MC     2.62
L-SC     1.42
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.17,-3.36,37.94
IT,11.79,-15.36,72.20
FINANCE,8.31,-17.41,60.02
BANKS,7.50,-13.79,60.22
MISC,6.12,-17.74,72.73
PAINTS,5.65,-15.82,38.65
ELECTRICAL,5.39,-2.35,50.86
HEALTHCARE,4.63,2.25,23.71
AUTO,4.41,-14.51,64.33


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2737907.0,25
XR,1079517.0,13
AR,1040212.0,10
X40,782490.0,10
X40N,592440.0,12
XY25,500850.0,8
OX40N,350700.0,11
SR,188581.0,2
X5K,138929.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2464958.0,26
M-SC,1617968.0,20
H-LC,1257425.0,20
H-MC,1167210.0,15
M-LC,419265.0,5
M-MC,327586.0,2
L-SC,209201.0,3
L-MC,56623.0,1
L-LC,48125.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,957800.0,7
M-SC,XY24,776314.0,7
H-SC,AR,520017.0,3
H-LC,X40,505862.0,6
H-MC,XY24,452728.0,4
H-SC,XR,368024.0,4
M-MC,XY24,327586.0,2
H-LC,X40N,293791.0,6
M-SC,XR,281649.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 46.0 seconds
